# L’analyse sémantique latente (LSA)

Le modèle vectoriel décrit par Gérard Salton dans son article [*A vector space model for automatic indexing*](https://dl.acm.org/doi/10.1145/361219.361220) publié en 1970, propose de représenter un document par un sac de mots, eux-mêmes réduits à leur racine, avant de les transformer en une matrice d’occurrences pondérée par TF-IDF qui peut servir de base à un calcul de similarité entre les documents.

Si la pondération TF-IDF permet de réintroduire une relation entre le terme et le document, elle échoue toutefois à rendre compte de sémantique. C’est ensuite en 1990 que Scott Deerwester et al. publient un papier sur une méthode brevetée deux ans plus tôt et intitulé : [*Indexing by Latent Semantic Analysis*](https://web.stanford.edu/class/linguist289/lsi.pdf). Leur approche se justifie par plusieurs aspects :

- **Sparsité :** La matrice TF-IDF est trop creuse et contient davantage les termes propres à chaque document que ceux qui les relient tous.
- **Synonymie :** Par extension, les termes ne sont pas reliés par une notion de synonymie mais uniquement par une notion de rareté.
- **Bruit :** Certains termes n’apparaissent que trop rarement dans un corpus et la pondération TF-IDF peut leur accorder trop d’importance.
- **Taille :** La très haute dimensionnalité de l’espace, en plus d’être majoritairement constitué de vide, peut être supérieure aux capacités pratiques de calcul des machines.

En trouvant une matrice de rang inférieur par décomposition, la LSA vise à capturer les relations latentes dans la matrice initiale en projetant les termes et les documents dans un espace réduit qui représente des concepts. Les résultats obtenus pourront à leur tour servir à améliorer la performance des modèles de classification ou de clustering.

## Une chaîne de traitement pour la LSA

Une chaîne de traitement envisageable passerait par les étapes successives :

1. Modélisation BoW
    - tokenisation
    - étiquetage morpho-syntaxique
    - racinisation (ou, mieux, lemmatisation)
2. Pondération TF-IDF (éventuellement normalisation)
3. Décomposition en valeurs singulières (SVD)
5. Analyse

## Décomposition en valeurs singulières

Partons de la situation où nous disposons d’une matrice TF-IDF normalisée avec la norme $L^2$ et constituée à partir d’un corpus de trois textes de Nietzsche préalablement lemmatisés :

In [ ]:
import pickle

with open("./data/fool.pickle", "rb") as f:
    data = pickle.load(f)

vocabulary = data['vocabulary']
matrix = data['matrix']

Visualisons-la dans un *data frame* avec *Pandas* :

In [ ]:
import pandas as pd

df = pd.DataFrame(matrix, columns=vocabulary)

display(df)

La décomposition SVD revient à considérer la matrice TF-IDF comme un produit de trois matrices plus simples :

$$
A = U \Sigma V^T
$$

Où :
- $U$ est une matrice unitaire constituée des vecteurs propres de $AA^T$
- $\Sigma$ une matrice des valeurs singulières de $A$
- et $V^T$ une matrice unitaire constituée des vecteurs propres de $A^TA$

Dans le cadre de la LSA, on va plutôt interpréter les matrices ainsi :

- $U$ pour les termes
- $\Sigma$ pour l’importance des concepts
- $V^T$ pour les documents

La décomposition se résoud en une passe avec la méthode `.linalg.svd()` de *Numpy* :

In [ ]:
import numpy as np

U, S, VT = np.linalg.svd(matrix)

La matrice $\Sigma$ contient désormais le vecteur des concepts. Comme il y a autant de concepts que de documents dans la matrice d’origine, le tout est de déterminer le nombre de concepts nécessaires pour expliquer un maximum la variance des données. La visualisation d’un **diagramme d’éboulis** peut aider à représenter le phénomène : à partir du moment où la pente s’arrête et qu’un coude se matérialise, nous pouvons en déduire que les concepts supplémentaires n’apportent pas d’explication significative à la variance.

Avant de regarder ce diagramme, il faut noter que les valeurs dans les matrices ne sont pas triés par ordre de grandeur. Or, si nous voulons visualiser un phénomène décroissant, nous devons au préalable trier les matrices sans perdre les indices d’assignation :

In [ ]:
# keep track of indices
indices = np.argsort(S)[::-1]

# sort S and the others as well
S = np.sort(S)[::-1]
U = U[:, indices]
VT = VT[indices, :]

Jetons à présent un œil au diagramme d’éboulis :

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(8, 6))
sns.lineplot(x=range(1, len(S) + 1), y=S, marker="o")

# title and labels
plt.title("Diagramme d’éboulis")
plt.xlabel("Concepts")
plt.ylabel("Valeurs singulières")

sns.despine()

plt.show()

La courbe décroissante semble parfaitement droite. Il n’y a en fait pas grand chose d’étonnant à cette curiosité comme on a normalisé notre matrice TF-IDF avec la norme $L^2$ avant la décomposition SVD. Le propre de la norme $L^2$ est de faire que les normes des vecteurs documents soient égales à 1.

Passons à une meilleure option : comparer la variance cumulative expliquée au seuil de 90 % :

In [ ]:
explained_variance = (S ** 2) / sum(S ** 2)
cumulative_variance = np.cumsum(explained_variance)

plt.figure(figsize=(8, 6))
sns.lineplot(x=range(1, len(S) + 1), y=cumulative_variance, marker="o")

plt.axhline(y=0.9, color="r", linestyle="--", label="90% Variance")

# title and labels
plt.title("Variance cumulative expliquée")
plt.xlabel("Concepts")
plt.ylabel("Variance Cumulative")

plt.legend()
sns.despine()

plt.show()

Selon ce diagramme, le nombre idéal de concepts à retenir se situe entre 2 et 3. Comme l’objectif est de réduire la dimensionnalité de notre jeu de données, nous nous contentons des deux premiers concepts.

## Projection des données dans la matrice réduite

Des trois concepts, nous n’avons retenu que les deux premiers au titre qu’ils expliquent 70 % de la variance totale. En pratique, nous ne devrions pas nous contenter d’un taux si bas mais ce résultat s’explique par le peu de concepts qui ont émergé des documents.

Constituons le sous-espace des concepts latents et projetons dedans les documents et les termes de notre étude :

In [ ]:
# number of concepts to keep
k = 2

# remove unwanted data
U_k = U[:, :k]
S_k = np.diag(S[:k])
VT_k = VT[:k, :]

# documents and terms are projected in the space of the concepts
reduced_matrix = np.dot(np.dot(U_k, S_k), VT_k)

L’objet `reduced_matrix` est maintenant une version approximée de la matrice originale. De la même manière, nous pouvons d’un côté projeter les documents dans l’espace des concepts et, de l’autre côté, les termes :

In [ ]:
document_matrix = np.dot(U_k, S_k)
term_matrix = np.dot(S_k, VT_k)

Ces deux projections nous servent à identifier visuellement des regroupements :

In [ ]:
# figure and subgraphs
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

fig.suptitle("Projection dans l’espace des concepts", fontsize=16)

# documents projected in the concept space
axes[0].scatter(document_matrix[:, 0], document_matrix[:, 1], color='blue')
axes[0].set_title("Documents")
axes[0].set_xlabel("Concept 1")
axes[0].set_ylabel("Concept 2")

# terms projected in the concept space
axes[1].scatter(term_matrix[0, :], term_matrix[1, :], color='red')
axes[1].set_title("Termes")
axes[1].set_xlabel("Concept 1")
axes[1].set_ylabel("Concept 2")

# annotations
for i, (x, y) in enumerate(zip(document_matrix[:, 0], document_matrix[:, 1])):
    axes[0].annotate(i, (x, y), textcoords="offset points", xytext=(0,10), ha='center', fontsize=10)

for i, (x, y) in enumerate(zip(term_matrix[0, :], term_matrix[1, :])):
    axes[1].annotate(vocabulary[i], (x, y), textcoords="offset points", xytext=(0,10), ha='center', fontsize=10)

plt.tight_layout()

sns.despine()

plt.show()

## Vers d’autres analyses

La LSA n’est qu’une étape, certes révélatrice, vers d’autres types d’analyses. En dévoilant une relation sémantique jusque-là invisible entre les documents et les termes d’un corpus nietzschéen réduit, elle nous permet désormais d’effectuer des calculs de distance ou de similarité, d’appliquer des méthodes de partitionnement pour les classer dans des catégories ou pour simplement les regrouper dans des ensembles homogènes.

Dans le contexte d’une recherche d’information, nous devrions d’abord vectoriser la requête d’un utilisateur en attribuant aux mots présents dans le vocabulaire la valeur de leur IDF. Si un mot apparaît plusieurs fois dans la requête, on peut multiplier son poids afin de refléter son importance. Une fois le vecteur de requête reconstitué, il doit ensuite être projeté dans l’espace des concepts à l’aide de la matrice $VT_k$. Il restera enfin à calculer la similarité cosinus entre ce vecteur projeté et les projections des documents ($U_k$), puis à analyser les résultats.